<a href=\"https://colab.research.google.com/github/acesur/GP-Lab1/blob/main/Lab1.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>

# Bayesian Regression Analysis of LIDAR Data\n\nThis notebook performs Bayesian regression analysis on synthetic LIDAR data:\n- Creates 221 LIDAR observations\n- Randomly selects 100 data points\n- Fits Bayesian quadratic regression model\n- Shows scatter plot with fitted model\n- Displays posterior distributions of parameters

In [ ]:
# Install required packages\n!pip install pymc arviz -q\nprint('📦 Packages installed successfully!')

In [ ]:
# Bayesian LIDAR Analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc as pm
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

print("🔄 Creating LIDAR dataset...")
# Create LIDAR data (221 observations)
range_vals = np.sort(np.random.uniform(390, 720, 221))
true_function = -0.5 * ((range_vals - 500) / 100)**2 + 2
noise = np.random.normal(0, 0.3, 221)
logratio = true_function + noise
lidar_data = pd.DataFrame({'range': range_vals, 'logratio': logratio})
print(f"✅ Full dataset: {len(lidar_data)} observations")

# Randomly select 100 points
sample_indices = np.random.choice(221, 100, replace=False)
X = lidar_data.iloc[sample_indices]['range'].values
y = lidar_data.iloc[sample_indices]['logratio'].values
X_scaled = (X - X.mean()) / X.std()
print(f"✅ Sample: {len(X)} observations selected")

print("\n🔄 Fitting Bayesian model...")
# Bayesian regression
with pm.Model() as model:
    intercept = pm.Normal('intercept', mu=0, sigma=2.5)
    beta1 = pm.Normal('beta1', mu=0, sigma=2.5)
    beta2 = pm.Normal('beta2', mu=0, sigma=2.5)
    sigma = pm.Exponential('sigma', lam=1)
    mu = intercept + beta1 * X_scaled + beta2 * X_scaled**2
    likelihood = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)
    trace = pm.sample(1000, tune=500, chains=2, random_seed=42, 
                     return_inferencedata=True, progressbar=True)

print("✅ Model fitted!")

# Generate predictions
X_plot = np.linspace(X.min(), X.max(), 100)
X_plot_scaled = (X_plot - X.mean()) / X.std()
posterior = trace.posterior
intercept_samples = posterior['intercept'].values.flatten()
beta1_samples = posterior['beta1'].values.flatten()
beta2_samples = posterior['beta2'].values.flatten()
sigma_samples = posterior['sigma'].values.flatten()

predictions = []
for i in range(len(intercept_samples)):
    pred = intercept_samples[i] + beta1_samples[i] * X_plot_scaled + beta2_samples[i] * X_plot_scaled**2
    predictions.append(pred)
predictions = np.array(predictions)
pred_mean = predictions.mean(axis=0)
pred_lower = np.percentile(predictions, 2.5, axis=0)
pred_upper = np.percentile(predictions, 97.5, axis=0)

print("\n📊 Creating plots...")

# PLOT 1: Scatter plot with fitted model
plt.figure(figsize=(10, 6))
plt.scatter(X, y, color='darkblue', alpha=0.7, s=50, label='Data points')
plt.plot(X_plot, pred_mean, color='red', linewidth=2, label='Posterior mean')
plt.fill_between(X_plot, pred_lower, pred_upper, alpha=0.3, color='red, 
                 label='95% credible interval')
plt.xlabel('Range')
plt.ylabel('Log Ratio')
plt.title('Bayesian Regression: LIDAR Data Analysis')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# PLOT 2: Parameter histograms
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Posterior Distributions of Model Parameters', fontsize=14)

params = [intercept_samples, beta1_samples, beta2_samples, sigma_samples]
param_names = ['Intercept', 'Beta1 (Linear)', 'Beta2 (Quadratic)', 'Sigma (Noise)']
colors = ['skyblue', 'lightgreen', 'orange', 'purple']

for i, (param, name, color) in enumerate(zip(params, param_names, colors)):
    ax = axes[i//2, i%2]
    ax.hist(param, bins=30, alpha=0.7, color=color, edgecolor='black')
    ax.axvline(param.mean(), color='red', linestyle='--', linewidth=2)
    ax.set_title(name)
    ax.set_xlabel('Parameter Value')
    ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("🎉 Analysis Complete!")
print(f"📈 Results Summary:")
print(f"   • Intercept: {intercept_samples.mean():.3f} ± {intercept_samples.std():.3f}")
print(f"   • Beta1: {beta1_samples.mean():.3f} ± {beta1_samples.std():.3f}")
print(f"   • Beta2: {beta2_samples.mean():.3f} ± {beta2_samples.std():.3f}")
print(f"   • Sigma: {sigma_samples.mean():.3f} ± {sigma_samples.std():.3f}")